In [1]:
# Import mlrun and create project instance
import mlrun
mlrun.set_env_from_file('../../cust_cs.env')

image = "mlrun/mlrun"
project_name = "monitored-model-runner-naive"
project = mlrun.get_or_create_project(project_name, context="./",user_project=True, allow_cross_project=True)

> 2025-11-05 15:35:46,384 [info] Project loaded successfully: {"project_name":"monitored-model-runner-naive-matanz"}


In [2]:
# Import tools
import pandas as pd
from sklearn.svm import SVC
import pickle
from sklearn.datasets import load_iris
from mlrun.features import Feature

In [3]:
# Train simple SVM model on Iris dataset, save it and reformat the DS as list
iris = load_iris()
clf = SVC()
clf.fit(iris.data, iris.target)
with open("SVM.pkl", "wb") as fh:
    pickle.dump(clf, fh)
iris_data = iris["data"].tolist()

# load the dataset again as a DF
iris = load_iris()
train_set = pd.DataFrame(
    iris["data"],
    columns=["sepal_length_cm", "sepal_width_cm", "petal_length_cm", "petal_width_cm"],
)

# Create a Model Artifact in the project using the trained model
model_name = "SVM"
model_artifact = project.log_model(
    model_name,
    model_file="SVM.pkl",
    training_set=train_set,
    framework="sklearn",
    outputs=[Feature(name="label")],
)

#### 5. Define your function and ModelRunnerStep

Define functions to all the edge cases

In [18]:
# Config the code path and the serving function sync and async
bouth_code_path = r"model_class_bouth.py"

function = project.set_function(func=bouth_code_path,image="mlrun/mlrun",kind="serving",name="bouth")

In [19]:
from mlrun.serving.states import ModelRunnerStep

model_runner_step = ModelRunnerStep(
    name="my_runner", model_selector="MyModelSelector",model_selector_parameters={"name":"my-selector"})

for i in range(100):
    model_runner_step.add_model(
        model_class="MyModel",
        endpoint_name=f"my-{i}-model",
        model_artifact=model_artifact,
        input_path="inputs.here",
        result_path="outputs",
        outputs=["label"],
        execution_mechanism="process_pool",
    )

#### 6. Build graphs to all the edge cases

description


In [20]:
async_graph_undefined = function.set_topology("router")

# async_graph_undefined.to("MyPreprocessStep").to(model_runner_step).to("MyEnrichStep").respond()
# async_graph_undefined.plot()

#### Run using mock

descteption

In [21]:
# # 4. create mock server and test it locally (no k8s / deployment)
# mock_server = function.to_mock_server()

In [22]:
# mock_server

In [23]:
# from random import choice
# from datetime import datetime
#
# iris_data = iris["data"].tolist()
# data_point = choice(iris_data)
# print(f"Data point:{data_point}")
#
# time_before = datetime.now()
# print("Before invoke:", time_before.strftime("%H:%M:%S.%f"))
# moke_response = mock_server.test(
#     "/",
#     body={
#         "models": None,
#         "inputs": [data_point, data_point],
#     },
# )
# time_after = datetime.now()
# print("After invoke:", time_after.strftime("%H:%M:%S.%f"))
# total_time = (time_after - time_before).total_seconds()
#


In [24]:
#moke_response

#### 7. Deploying Your Function

Running this cell will deploy your serving function to the cluster. This also deploys the real-time monitoring functions for your project, which are configured to track the serving function's performance and detect model drift.

In [25]:
function.deploy()

> 2025-11-05 15:43:36,435 [info] Starting remote function deploy
> 2025-11-05 15:43:36,439 [warning] POST https://mlrun-api.default-tenant.app.cust-cs-il.iguazio-cd0.com/api/v1/projects/monitored-model-runner-naive-matanz/nuclio/bouth/deploy request failed, http retries disabled for POST method.: {"exception_message":"Object of type ModelArtifact is not JSON serializable","exception_type":"<class 'TypeError'>","max_retries":3,"retry_count":0,"retry_interval":1}


TypeError: Object of type ModelArtifact is not JSON serializable

In [26]:
from random import choice
from datetime import datetime

iris_data = iris["data"].tolist()
data_point = choice(iris_data)
print(f"Data point:{data_point}")

time_before = datetime.now()
print("Before invoke:", time_before.strftime("%H:%M:%S.%f"))
moke_response = function.invoke(
    "/",
    body={
        "models": None,
        "inputs": [data_point, data_point],
    },
)
time_after = datetime.now()
print("After invoke:", time_after.strftime("%H:%M:%S.%f"))
total_time = (time_after - time_before).total_seconds()

Data point:[5.0, 2.0, 3.5, 1.0]
Before invoke: 15:43:36.932212
> 2025-11-05 15:43:37,569 [warning] Function is in the imported state


RuntimeError: bad function response 404: default backend - 404

In [13]:
print(f"Total time: {total_time}")

Total time: 39.172358


In [14]:
#deploy_response